# 策略持仓权重管理
---

In [1]:
# 设置 Redis 链接信息，用于存储持仓权重信息
import os
os.environ['RWC_REDIS_URL'] = 'redis://***:***@***:6379/0'

# 也可以使用 dotenv 来设置环境变量
from dotenv import load_dotenv
# load_dotenv(r'path/to/.env', override=True)
load_dotenv(r"D:\ZB\git_repo\waditu\czsc\examples\test_offline\.env", override=True)

True

### 一、写入策略持仓权重


策略持仓权重样例数据可以从飞书文档中下载：

In [3]:
import czsc
import redis
import pandas as pd

# 创建 BlockingConnectionPool 连接池，必须设置 decode_responses=True 参数，否则返回的数据是 bytes 类型
connection_pool = redis.BlockingConnectionPool.from_url(os.getenv("RWC_REDIS_URL"), decode_responses=True)


# 策略写入阶段，必须设置 send_heartbeat=True 参数，这样可以在写入数据时自动更新策略的最新写入时间
rwc = czsc.RedisWeightsClient('TestStrategy', connection_pool=connection_pool, key_prefix='Weights', send_heartbeat=True)

# 首次写入，建议设置一些策略元数据
rwc.set_metadata(description='测试策略：仅用于读写redis测试', base_freq='日线', author='测试A', outsample_sdt='20220101')

# 查看策略元数据
print("策略元数据：", rwc.metadata)

# 写入策略持仓权重，样例数据下载：https://s0cqcxuy3p.feishu.cn/wiki/Pf1fw1woQi4iJikbKJmcYToznxb
dfw = pd.read_feather(r"C:\Users\zengb\Downloads\weight_example.feather")
print("持仓权重数据：\n", dfw.head())

# on_bar 模式下按单条数据写入，效率更高，可以实时更新持仓权重
# rwc.publish(**dfw.iloc[0].to_dict())

# 盘后或者定时执行的策略，批量写入可以提高效率
rwc.publish_dataframe(dfw, overwrite=False, batch_size=10000)

2024-03-07 20:49:38.448 | INFO     | czsc.traders.rwc:__init__:62 - TestStrategy Weights: 使用传入的 redis 连接池
2024-03-07 20:49:39.819 | WARNING  | czsc.traders.rwc:set_metadata:85 - 已存在 TestStrategy 的元数据，如需覆盖请设置 overwrite=True


策略元数据： {'name': 'TestStrategy', 'base_freq': '日线', 'key_prefix': 'Weights', 'description': '测试策略：仅用于读写redis测试', 'author': '测试A', 'outsample_sdt': '20220101', 'update_time': '2024-03-07 20:49:05', 'kwargs': '{}'}


2024-03-07 20:49:40.338 | INFO     | czsc.traders.rwc:publish_dataframe:172 - 输入数据中有 3938513 条权重信号


持仓权重数据：
                    dt    symbol  weight  price
0 2018-03-26 09:01:00  SEsc9001     0.0  442.9
1 2018-03-26 09:02:00  SEsc9001     0.0  441.2
2 2018-03-26 09:03:00  SEsc9001     0.0  440.4
3 2018-03-26 09:04:00  SEsc9001     0.0  439.5
4 2018-03-26 09:05:00  SEsc9001     0.0  440.7


2024-03-07 20:49:41.165 | INFO     | czsc.traders.rwc:publish_dataframe:182 - 去除单个品种下相邻时间权重相同的数据后，剩余 29906 条权重信号
2024-03-07 20:49:42.327 | INFO     | czsc.traders.rwc:publish_dataframe:199 - 不允许重复写入，已过滤 29906 条重复信号
2024-03-07 20:49:42.518 | INFO     | czsc.traders.rwc:update_last:105 - 更新 Weights:LAST:TestStrategy 的 last 时间


0

### 二、读取策略持仓权重



### 三、删除 redis 中的策略

In [8]:
import os
import czsc

# 获取 RWC_REDIS_URL 环境变量，用于连接 Redis
# redis_url = os.getenv("RWC_REDIS_URL")
# czsc.clear_strategy(strategy_name="TestStrategy", redis_url=redis_url, key_prefix="Weights")

# redis_url 参数可以省略，如果省略则使用环境变量 RWC_REDIS_URL
czsc.clear_strategy(strategy_name="TestStrategy", key_prefix="Weights")

2024-03-07 20:41:12.209 | INFO     | czsc.traders.rwc:__init__:67 - STK008pro_3 WeightsB: 使用环境变量 RWC_REDIS_URL 创建 redis 连接池
2024-03-07 20:41:17.818 | INFO     | czsc.traders.rwc:clear_all:262 - STK008pro_3 删除了 16714 条记录
